In [8]:
#This was not fully tested b/c it kept stopping in colab. Probably because of high RAM usage.
!pip install -q transformers einops accelerate bitsandbytes
!pip install --upgrade protobuf
!pip install bitsandbytes==0.41.1
!pip install torch
!pip freeze -l > /content/sample_data/requirements.txt

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
import torch

# model_name = "meta-llama/Llama-2-3b-hf" #not exist
model_name = "meta-llama/Llama-2-7b-hf"  # Use the 7B model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True, force_download=True)

# Load model without quantization on CPU
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map={"": "cpu"},  # Force the model to run on CPU
                                             use_auth_token=True,
                                             force_download=True)



/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Sample input: medical visit history in text
medical_text = """
Patient visited the clinic on October 15, 2023. They were seen by Dr. Smith at Erlanger Health.
The patient received treatment for diabetes and hypertension. A follow-up was scheduled for December 10, 2023.
"""

# Define a prompt that asks the model to extract structured data
prompt = """
Extract the following information from the text:
1. Date of treatment
2. Medical provider
3. Treatment received
Input text: {}
""".format(medical_text)

# Tokenize the input text for the model
inputs = tokenizer(prompt, return_tensors="pt")  # No need to move tensors to GPU

# Generate the model output (run on CPU)
with torch.no_grad():  # No gradients needed for inference
    outputs = model.generate(**inputs, max_length=512)

# Decode the generated output
extracted_data = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the extracted data
print("Extracted Data:\n", extracted_data)